## Introduction
This notebook is designed to guide you through the process of building an AI agent using Langchain. This will actually put it all together from the previous lessons

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

#### Setup custom tools
Let's add the two custom tools we defined in the last notebook: url retriever and summarizer

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools import tool

@tool
def fetch_url_content(url: str) -> str:
    """Use this tool to fetch content of a URL"""
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic import ChatAnthropic

@tool
def summarize_content(content: str) -> str:
    """Always use this tool to generate the summary of a text based content. This model is optimized for summarization"""
    system_message = "Please provide a concise summary for the content"
    human_message = "Content: " + str(content)
    
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=human_message),
    ]
    model = ChatAnthropic(model="claude-3-haiku-20240229")
    parser = StrOutputParser()
    
    chain = model | parser
    summary = chain.invoke(messages)
    return summary

In [4]:
# Let's add both of these custom tools to the tools that we want to provide to our agent
tools = [fetch_url_content, summarize_content]

In [5]:
# Let's add the model we want to use for the agent. This is the LLM that acts as the brain of the agent. It is a good idea to use the highest quality model you can use.
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

#### Let's add the agent now that can use the tools
We will use a simple ReAct agent from Langgraph. At the time of this writing, Langchain is trying to get devs to start using Langgraph for agent development and away from the existing agent types from the core library

In [6]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [7]:
agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

{'messages': [HumanMessage(content='hi!', id='d18574e4-eb17-4a4c-97c6-242fbfc4f410'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 86, 'total_tokens': 96}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fcf143c-c288-4266-a18f-f31a775fb96f-0', usage_metadata={'input_tokens': 86, 'output_tokens': 10, 'total_tokens': 96})]}

In [ ]:
# Now let's ask it to summarize the content of a URL
# The speed on LLMs is meaured in terms of the tokens per minute it can produce. Since this is a large blog post, the LLM will take a while to read and summarize it
response = agent_executor.invoke({"messages": [HumanMessage(content="can you please summarize the content of this url https://lilianweng.github.io/posts/2023-06-23-agent/")]})
response["messages"]